# 목표
프로젝트의 목표는 교통사고 데이터(RoadSense)를 기반으로 사고 심각도(Gravity)를 예측하는 모델을 개발하는 것입니다.     
최종적으로 사고 심각도를 조기에 예측할 수 있는 모델을 통해 교통 안전 개선 및 사고 예방 정책 수립에 데이터 기반 인사이트를 제공하는 것을 목표로 합니다.

## DataPreprocessing의 목표
- 중복 데이터를 처리한다.
- Test를 기준으로 Train 데이터 셋의 type과 정합성을 일치시킨다.

### 데이터 셋 
실제 교통사고 데이터를 기반으로 교통사고 심각도 예측 Kaggle 대회용으로 수정된 데이터셋을 사용했습니다.    
사고 정보, 도로 정보, 차량 정보, 인적 요인 데이터셋으로 구성되어 있습니다.

In [1]:
# 라이브러리

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import re
import string
from datetime import datetime

In [2]:
# 1. 데이터 로드 및 변수 저장

accidents_train = pd.read_csv('./data/accidents_train.csv')
places_train = pd.read_csv('./data/places_train.csv')
users_train = pd.read_csv('./data/users_train.csv')
vehicles_train = pd.read_csv('./data/vehicles_train.csv')

accidents_test = pd.read_csv('./data/accidents_test.csv')
places_test = pd.read_csv('./data/places_test.csv')
users_test = pd.read_csv('./data/users_test.csv')
vehicles_test = pd.read_csv('./data/vehicles_test.csv')

# 1. Accidents

Accidents 데이터는 전체 분석의 기준이 되는 핵심 테이블이다.    
본 파트에서는 test 데이터를 구조 기준(reference)으로 삼아 train 데이터의 중복 및 자료형 정합성을 단계적으로 정리한다.

진행 순서는 다음과 같다.

- Test 데이터 구조 및 중복 확인
- 데이터 타입 기준 정의
- Train–Test 간 데이터 타입 정합성 맞추기
- Train 데이터 중복 처리

## 1) Test

In [3]:
# 데이터 확인
accidents_test.head(3)

,AccidentId,Date,Hour,Light,Department,Commune,InAgglomeration,IntersectionType,Weather,CollisionType,PostalAddress,GPSCode,Latitude,Longitude
0,201800000005,2018-06-26,16:05:00,Daylight,590,477,Yes,NoIntersection,Normal,2Vehicles-Side,72 rue Victor Hugo,M,50.51607,2.51607
1,201800000007,2018-09-26,00:40:00,NightStreelightsOn,590,133,Yes,NoIntersection,Normal,Other,4 route de camphin,M,50.52211,2.52211
2,201800000035,2018-03-16,21:15:00,NightNoStreetLight,590,581,No,NoIntersection,LightRain,Other,7 rue du pont de pierre,M,50.71101,2.71101


In [4]:
# 데이터 정보 확인
accidents_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5779 entries, 0 to 5778
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   AccidentId        5779 non-null   int64  
 1   Date              5779 non-null   object 
 2   Hour              5779 non-null   object 
 3   Light             5779 non-null   object 
 4   Department        5779 non-null   int64  
 5   Commune           5779 non-null   int64  
 6   InAgglomeration   5779 non-null   object 
 7   IntersectionType  5779 non-null   object 
 8   Weather           5779 non-null   object 
 9   CollisionType     5779 non-null   object 
 10  PostalAddress     5721 non-null   object 
 11  GPSCode           5774 non-null   object 
 12  Latitude          5577 non-null   float64
 13  Longitude         5577 non-null   float64
dtypes: float64(2), int64(3), object(9)
memory usage: 632.2+ KB


In [5]:
# 중복 데이터 갯수 확인
print(f"중복된 데이터 갯수: {accidents_test.duplicated().sum()}")

중복된 데이터 갯수: 0


In [6]:
# AccidentId 중복 데이터 갯수 확인
print(f"'AccidentId'기준 중복된 데이터 갯수: {accidents_test['AccidentId'].duplicated().sum()}")

'AccidentId'기준 중복된 데이터 갯수: 0


### 데이터 타입 기준 정의

Accidents 데이터는 test 데이터를 기준으로 train 데이터의 정합성을 맞추기 위해 각 컬럼의 의미에 따라 데이터 타입 기준을 먼저 정의한다.

- AccidentId, Department, Commune    
  숫자 형태이지만 계산 대상이 아닌 식별자 및 코드 값이므로 문자열 타입으로 처리한다.

- Date, Hour    
  시간 정보를 포함한 컬럼으로 이후 시간 기반 파생변수 생성을 고려하여 datetime / time 형태로 변환 가능한 기준 컬럼으로 설정한다.

- Latitude, Longitude    
  위치 정보를 나타내는 수치형 컬럼으로 float 타입을 유지한다.

- PostalAddress, GPSCode  
  텍스트 정보 컬럼으로 문자열 타입을 유지한다.

In [7]:
# 데이터 타입 기준 정의
Accidents_type_reference = {
    "AccidentId": "int64",
    "Department": "string",
    "Commune": "string",
    "Light": "string",
    "InAgglomeration": "string",
    "IntersectionType": "string",
    "Weather": "string",
    "CollisionType": "string",
    "PostalAddress": "string",
    "GPSCode": "string",
    "Latitude": "float64",
    "Longitude": "float64",
}

accidents_test = accidents_test.astype(Accidents_type_reference)

In [8]:
# 날짜 / 시간 컬럼 파싱
accidents_test["Date"] = pd.to_datetime(accidents_test["Date"], errors="coerce")
accidents_test["Hour"] = pd.to_timedelta(accidents_test["Hour"], errors="coerce")

In [9]:
# 변경된 info() 확인
accidents_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5779 entries, 0 to 5778
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype          
---  ------            --------------  -----          
 0   AccidentId        5779 non-null   int64          
 1   Date              5779 non-null   datetime64[ns] 
 2   Hour              5779 non-null   timedelta64[ns]
 3   Light             5779 non-null   string         
 4   Department        5779 non-null   string         
 5   Commune           5779 non-null   string         
 6   InAgglomeration   5779 non-null   string         
 7   IntersectionType  5779 non-null   string         
 8   Weather           5779 non-null   string         
 9   CollisionType     5779 non-null   string         
 10  PostalAddress     5721 non-null   string         
 11  GPSCode           5774 non-null   string         
 12  Latitude          5577 non-null   float64        
 13  Longitude         5577 non-null   float64        
dtypes: datet

## 2) Train

In [10]:
# 데이터 확인
accidents_train.head(3)

,AccidentId,Gravity,Date,Hour,Light,Department,Commune,InAgglomeration,IntersectionType,Weather,CollisionType,PostalAddress,GPSCode,Latitude,Longitude
0,201800000001,NonLethal,24/01/2018,15:05:00,daylight,590,5,No,Y-type,Normal,2Vehicles-BehindVehicles-Frontal,route des Ansereuilles,M,50.555225,2.556955
1,201800000002,NonLethal,Feb 12 2018,10:15:00,Daylight,590,11,Yes,Square,VERYGOOD,NoCollision,Place du général de Gaul,M,50.529369,2.529158
2,201800000003,NonLethal,04-Mar-18,11:35:00:20,daylight,590,477,Yes,T-type,Normal,NoCollision,Rue nationale,M,50.510923,2.512682


In [11]:
# 데이터 정보 확인
accidents_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52004 entries, 0 to 52003
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   AccidentId        52004 non-null  object 
 1   Gravity           47822 non-null  object 
 2   Date              52004 non-null  object 
 3   Hour              52004 non-null  object 
 4   Light             52004 non-null  object 
 5   Department        52004 non-null  int64  
 6   Commune           52004 non-null  int64  
 7   InAgglomeration   52004 non-null  object 
 8   IntersectionType  52004 non-null  object 
 9   Weather           51999 non-null  object 
 10  CollisionType     52002 non-null  object 
 11  PostalAddress     51507 non-null  object 
 12  GPSCode           51959 non-null  object 
 13  Latitude          50383 non-null  float64
 14  Longitude         50383 non-null  float64
dtypes: float64(2), int64(2), object(11)
memory usage: 6.0+ MB


### 데이터 정합성 맞추기

앞선 단계에서 test 데이터를 기준으로 각 컬럼의 데이터 타입과 처리 원칙을 정의하였다.

본 단계에서는 해당 기준을 그대로 적용하여 train 데이터의 컬럼 타입을 일관되게 맞춘다.    
이 과정을 통해 train–test 간 스키마 불일치로 인한 병합 오류 및 모델 입력 오류를 사전에 방지하고 이후 전처리 및 모델링 단계의 재현성을 확보한다.

#### AccidentId 컬럼
AccidentId는 아래와 같은 데이터 형식을 포함한다.
- '-' 하이픈 포함
- 공백 포함

위 조건을 test와 동일하게 하이픈이 포함되지 않은 숫자만으로 치환한다.

In [12]:
# AccidentId 데이터 확인
accidents_train["AccidentId"].head(20)

0       201800000001
1       201800000002
2       201800000003
3       201800000004
4       201800000006
5       201800000008
6       201800000009
7       201800000010
8      201800000011 
9       201800000012
10      201800000013
11     20-1800000014
12      201800000015
13      201800000016
14      201800000017
15      201800000018
16      201800000019
17      201800000020
18      201800000021
19      201800000022
Name: AccidentId, dtype: object

In [13]:
# 하이픈 제거하기
accidents_train["AccidentId"] = accidents_train["AccidentId"].str.replace("-", "", regex = False)

In [14]:
# 공백 제거
accidents_train["AccidentId"] = accidents_train["AccidentId"].str.replace(" ", "", regex=False)

In [15]:
# AccidentId 데이터 확인
accidents_train["AccidentId"].head(20)

0     201800000001
1     201800000002
2     201800000003
3     201800000004
4     201800000006
5     201800000008
6     201800000009
7     201800000010
8     201800000011
9     201800000012
10    201800000013
11    201800000014
12    201800000015
13    201800000016
14    201800000017
15    201800000018
16    201800000019
17    201800000020
18    201800000021
19    201800000022
Name: AccidentId, dtype: object

#### Date 컬럼
Date는 아래와 같은 데이터 형식을 포함한다.
- 약어
- 구분자 '/', ' ', '-' 등 
- 년-월-일 형식이 섞여있음
- 시간데이터 혼재

위 조건을 test와 동일하게 YYYY-MM-DD 로 치환한다.

In [16]:
# Date 데이터 조회
accidents_train["Date"].head(20)

0              24/01/2018
1             Feb 12 2018
2               04-Mar-18
3             May 05 2018
4               23-Sep-18
5     2018-11-30 00:00:00
6             Feb 18 2018
7              19/03/2018
8              28/05/2018
9             May 31 2018
10             15/06/2018
11    2018-07-19 00:00:00
12    2018-10-31 00:00:00
13            Jan 15 2018
14    2018-02-09 00:00:00
15             07/04/2018
16              17-Apr-18
17              27-May-18
18    2018-06-12 00:00:00
19              12-Jul-18
Name: Date, dtype: object

**약어 수정**

In [17]:
# 약어 문자 숫자와 매핑
month_map = {
    'Jan':'01', 'Feb':'02', 'Mar':'03', 'Apr':'04',
    'May':'05', 'Jun':'06', 'Jul':'07', 'Aug':'08',
    'Sep':'09', 'Oct':'10', 'Nov':'11', 'Dec':'12'
}

In [18]:
# 약어 문자 숫자로 변환
accidents_train['Date'] = accidents_train['Date'].replace(month_map, regex=True)

**시간 미포함 데이터**
- 현 데이터는 2018년도 기준 데이터
- 구분자 '-'로 통일
- 시간 미포함 데이터는 맨뒤 날짜 포멧이 년도로 확인됨 (18 또는 2018)
- YYYY-MM-DD 형식을 맞추기위해 년도를 먼저 찾아서 2018년도 포맷을 맞춘다.
- 월/일 을 찾아서 YYYY-MM-DD 형식을 맞춰준다.

In [19]:
# 시간 미포함 mask 저장
mask_no_time = ~accidents_train["Date"].str.contains(r"\d{2}:\d{2}:\d{2}", na=False)

In [20]:
# 구분자 통일

accidents_train.loc[mask_no_time, "Date"] = (
    accidents_train.loc[mask_no_time, "Date"]
    .astype(str)
    .str.replace(r"[\/ ]", "-", regex=True)
)

In [21]:
# 시간 미포함 & '년도'가 맨뒤가 아닌 경우
accidents_train.loc[
    mask_no_time & ~accidents_train["Date"].astype(str).str.contains(r"(?:2018|18)$", na=False),"Date"
    ]

Series([], Name: Date, dtype: object)

In [22]:
# 18년도로 되어있는 년도를 2018로 수정

mask_18 = mask_no_time & accidents_train["Date"].str.contains(r"(?<!20)18$", na=False)

accidents_train.loc[mask_18, "Date"] = (
    accidents_train.loc[mask_18, "Date"].str.replace(r"18$", "2018", regex = True)
)

In [23]:
# 2018년도를 맨앞으로 이동하기
accidents_train.loc[mask_no_time, "Date"] = (
    accidents_train.loc[mask_no_time, "Date"].str.replace(r"^(\d{2})-(\d{2})-(2018)$", 
                                                          r"\3-\1-\2",
                                                          regex=True)
)

In [24]:
# 시간 미포함 기준 '월'이 12초과인 경우 mask 저장
mask_invalid_month = (
    mask_no_time &
    (accidents_train["Date"].str.extract(r"^2018-(\d{2})-(\d{2})")[0].astype(int) > 12)
)

In [25]:
# '월' 12 초과 자리 교체
accidents_train.loc[mask_invalid_month, "Date"] = (
    accidents_train.loc[mask_invalid_month, "Date"].str.replace(r"^2018-(\d{2})-(\d{2})$",
                                                                r"2018-\2-\1",
                                                                regex=True)
)

**시간 데이터 삭제**

In [26]:
# 00:00:00 포맷 삭제하기

accidents_train["Date"] = (accidents_train["Date"].astype(str).str.replace(r"\s+\d{2}:\d{2}:\d{2}$", 
                                                                           "", 
                                                                           regex=True)
)

In [27]:
# 최종 변환 확인
accidents_train["Date"].head(20)

0     2018-01-24
1     2018-02-12
2     2018-04-03
3     2018-05-05
4     2018-09-23
5     2018-11-30
6     2018-02-18
7     2018-03-19
8     2018-05-28
9     2018-05-31
10    2018-06-15
11    2018-07-19
12    2018-10-31
13    2018-01-15
14    2018-02-09
15    2018-07-04
16    2018-04-17
17    2018-05-27
18    2018-06-12
19    2018-12-07
Name: Date, dtype: object

#### Hour 컬럼

Hour는 아래와 같은 데이터 형식을 포함한다.
- 초 이상의 포맷 (00:00:00:00)
- PM, AM 포맷

위 조건을 test와 동일한 24시간 기준'%H:%M:%S'로 치환한다.

In [28]:
# Hour 데이터 조회
accidents_train["Hour"].head(20)

0        15:05:00
1        10:15:00
2     11:35:00:20
3         5:35 PM
4     06:30:00:58
5        17:15:00
6         3:57 PM
7     15:30:00:44
8        18:30:00
9     04:30:00:39
10       08:45:00
11    10:22:00:57
12       19:45:00
13       07:25:00
14        3:15 PM
15    11:45:00:35
16       16:10:00
17       16:00:00
18        5:45 PM
19       17:00:00
Name: Hour, dtype: object

In [29]:
#  콜론이 3개 이상 mask 생성
mask_colons = accidents_train["Hour"].str.count(":") > 2

In [30]:
# 00:00:00 포맷 맞추기
accidents_train.loc[mask_colons, "Hour"] = (
    accidents_train.loc[mask_colons, "Hour"].str.split(":").str[:3].str.join(":")
)

In [31]:
# AM / PM 을 24시간제로 통일하기
accidents_train["Hour"] = pd.to_datetime(accidents_train["Hour"],errors="coerce").dt.strftime("%H:%M:%S")

C:\Users\nini\AppData\Local\Temp\ipykernel_12472\1341841227.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  accidents_train["Hour"] = pd.to_datetime(accidents_train["Hour"],errors="coerce").dt.strftime("%H:%M:%S")


In [32]:
# 최종 변환 확인
accidents_train["Hour"].head(20)

0     15:05:00
1     10:15:00
2     11:35:00
3     17:35:00
4     06:30:00
5     17:15:00
6     15:57:00
7     15:30:00
8     18:30:00
9     04:30:00
10    08:45:00
11    10:22:00
12    19:45:00
13    07:25:00
14    15:15:00
15    11:45:00
16    16:10:00
17    16:00:00
18    17:45:00
19    17:00:00
Name: Hour, dtype: object

#### Light 컬럼

Light는 아래와 같은 데이터 형식을 포함한다.
- 대소문자 표기 규칙의 정합성 불일치

위 조건을 test와 동일한 5개의 포맷으로 변경한다.
- 'Daylight', 'NightStreelightsOn', 'NightNoStreetLight', 'TwilightOrDawn', NightStreelightsOff'

In [33]:
# Light 데이터 조회
accidents_train["Light"].unique()

array(['daylight', 'Daylight', 'DAYLIGHT', 'TwilightOrDawn',
       'NIGHTSTREELIGHTSON', 'NightNoStreetLight', 'TWILIGHTORDAWN',
       'twilightordawn', 'nightstreelightson', 'NIGHTNOSTREETLIGHT',
       'NightStreelightsOn', 'nightnostreetlight', 'NIGHTSTREELIGHTSOFF',
       'nightstreelightsoff', 'NightStreelightsOff'], dtype=object)

In [34]:
# Light 모두 소문자로 변환하기
accidents_train["Light"] = accidents_train["Light"].str.lower()

In [35]:
# 매핑 테이블
light_mapping = {
    "daylight": "Daylight",
    "twilightordawn": "TwilightOrDawn",
    "nightstreelightson": "NightStreelightsOn",
    "nightnostreetlight": "NightNoStreetLight",
    "nightstreelightsoff": "NightStreelightsOff"
}

In [36]:
# 매핑 적용하기
accidents_train["Light"] = accidents_train["Light"].map(light_mapping)

In [37]:
# 최종 결과
accidents_train["Light"].unique()

array(['Daylight', 'TwilightOrDawn', 'NightStreelightsOn',
       'NightNoStreetLight', 'NightStreelightsOff'], dtype=object)

#### Weather 컬럼

Weather는 아래와 같은 데이터 형식을 포함한다.
- 대소문자 표기 규칙의 정합성 불일치

위 조건을 test와 동일한 포맷으로 변경한다.

In [38]:
# weather 데이터 조회
accidents_train["Weather"].unique()

array(['Normal', 'VERYGOOD', 'LightRain', 'FOGORSMOKE', 'NORMAL',
       'SnowOrHail', 'normal', 'snoworhail', 'LIGHTRAIN', 'Overcast',
       'verygood', 'VeryGood', 'lightrain', 'Other', 'heavyrain',
       'SNOWORHAIL', 'OTHER', 'HEAVYRAIN', 'StrongWindOrStorm',
       'overcast', 'OVERCAST', 'fogorsmoke', 'other', 'FogOrSmoke',
       'HeavyRain', 'strongwindorstorm', 'STRONGWINDORSTORM', nan],
      dtype=object)

In [39]:
# Weather 모두 소문자로 변환
accidents_train["Weather"] = accidents_train["Weather"].str.lower()

In [40]:
# 매핑 테이블
Weather_mapping = {
    "normal": "Normal",
    "verygood": "VeryGood",
    "lightrain": "LightRain",
    "fogorsmoke": "FogOrSmoke",
    "snoworhail": "SnowOrHail",
    "overcast": "Overcast",
    "other": "Other",
    "heavyrain": "HeavyRain",
    "strongwindorstorm": "StrongWindOrStorm"
}

In [41]:
# 매핑 적용하기
accidents_train["Weather"] = accidents_train["Weather"].map(Weather_mapping)

In [42]:
accidents_train["Weather"].unique()

array(['Normal', 'VeryGood', 'LightRain', 'FogOrSmoke', 'SnowOrHail',
       'Overcast', 'Other', 'HeavyRain', 'StrongWindOrStorm', nan],
      dtype=object)

### 중복데이터 확인

In [43]:
# 중복 데이터 갯수 확인
print(f"중복된 데이터 갯수: {accidents_train.duplicated().sum()}")

중복된 데이터 갯수: 0


In [44]:
# AccidentId 기준 중복 데이터 갯수 확인
print(f"'AccidentId'기준 중복된 데이터 갯수: {accidents_train['AccidentId'].duplicated().sum()}")

'AccidentId'기준 중복된 데이터 갯수: 0


### 데이터 타입 기준 정리
- Test 와 동일하게 데이터 타입을 정리한다.

In [45]:
# 데이터 타입 기준 Test와 동일
Accidents_type_reference = {
    "AccidentId": "int64",
    "Department": "string",
    "Commune": "string",
    "Light": "string",
    "InAgglomeration": "string",
    "IntersectionType": "string",
    "Weather": "string",
    "CollisionType": "string",
    "PostalAddress": "string",
    "GPSCode": "string",
    "Latitude": "float64",
    "Longitude": "float64",
}

accidents_train = accidents_train.astype(Accidents_type_reference)

In [46]:
# 날짜 / 시간 컬럼 파싱
accidents_train["Date"] = pd.to_datetime(accidents_train["Date"], errors="coerce")
accidents_train["Hour"] = pd.to_timedelta(accidents_train["Hour"], errors="coerce")

In [47]:
# 변경된 info() 확인
accidents_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52004 entries, 0 to 52003
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype          
---  ------            --------------  -----          
 0   AccidentId        52004 non-null  int64          
 1   Gravity           47822 non-null  object         
 2   Date              52004 non-null  datetime64[ns] 
 3   Hour              52004 non-null  timedelta64[ns]
 4   Light             52004 non-null  string         
 5   Department        52004 non-null  string         
 6   Commune           52004 non-null  string         
 7   InAgglomeration   52004 non-null  string         
 8   IntersectionType  52004 non-null  string         
 9   Weather           51999 non-null  string         
 10  CollisionType     52002 non-null  string         
 11  PostalAddress     51507 non-null  string         
 12  GPSCode           51959 non-null  string         
 13  Latitude          50383 non-null  float64        
 14  Longit

# 2. Places

Places 데이터는 사고장소의 정보를 담고 있는 테이블이다.    
동일하게 test 데이터를 구조 기준(reference)으로 삼아 train 데이터의 중복 및 자료형 정합성을 단계적으로 정리한다.

진행 순서는 다음과 같다.

- Test 데이터 구조 및 중복 확인
- 데이터 타입 기준 정의
- Train–Test 간 데이터 타입 정합성 맞추기
- Train 데이터 중복 처리

## 1) Test

In [48]:
# 데이터 확인
places_test.head(3)

,AccidentId,RoadType,RoadNumber,RoadSecNumber,RoadLetter,Circulation,LaneNumber,SpecialLane,Slope,RoadMarkerId,RoadMarkerDistance,Layout,StripWidth,LaneWidth,SurfaceCondition,Infrastructure,Localization,SchoolNear
0,201800000005,Communal,NaN,NaN,NaN,OneWay,1.0,0,Flat,NaN,NaN,Straight,NaN,NaN,Normal,Unknown,Lane,0.0
1,201800000007,Departamental,41.0,NaN,D,TwoWay,2.0,0,Flat,16.0,500.0,Straight,NaN,NaN,Normal,Unknown,Shoulder,0.0
2,201800000035,Departamental,10.0,NaN,NaN,TwoWay,2.0,0,Flat,13.0,400.0,Straight,NaN,NaN,Wet,Unknown,Shoulder,0.0


In [49]:
# 데이터 정보 확인
places_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5779 entries, 0 to 5778
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   AccidentId          5779 non-null   int64  
 1   RoadType            5779 non-null   object 
 2   RoadNumber          3549 non-null   float64
 3   RoadSecNumber       18 non-null     float64
 4   RoadLetter          265 non-null    object 
 5   Circulation         5740 non-null   object 
 6   LaneNumber          5728 non-null   float64
 7   SpecialLane         5721 non-null   object 
 8   Slope               5738 non-null   object 
 9   RoadMarkerId        4204 non-null   float64
 10  RoadMarkerDistance  4169 non-null   float64
 11  Layout              5741 non-null   object 
 12  StripWidth          1310 non-null   float64
 13  LaneWidth           1356 non-null   float64
 14  SurfaceCondition    5735 non-null   object 
 15  Infrastructure      5730 non-null   object 
 16  Locali

In [50]:
# 중복 데이터 갯수 확인
print(f"중복된 데이터 갯수: {places_test.duplicated().sum()}")

중복된 데이터 갯수: 0


In [51]:
# AccidentId 기준 중복 데이터 갯수 확인
print(f"'AccidentId'기준 중복된 데이터 갯수: {accidents_test['AccidentId'].duplicated().sum()}")

'AccidentId'기준 중복된 데이터 갯수: 0


### 데이터 타입 기준 정의

- AccidentId 및 도로 번호/마커 ID 등은 식별자·코드 성격이므로 문자열로 처리하였다.
- 도로 폭/거리(StripWidth, LaneWidth, RoadMarkerDistance)는 연속형 수치 의미가 있어 float로 유지하였다.
- LaneWidth는 차선의 갯수로 정수형 Int64로 변환하였다.
- 나머지 도로 속성 컬럼은 범주형으로 문자열로 통일하였다.

In [52]:
# 데이터 타입 기준 정의
places_type_reference = {
    "AccidentId": "string",

    # 범주형
    "RoadType": "string",
    "RoadLetter": "string",
    "Circulation": "string",
    "SpecialLane": "string",
    "Slope": "string",
    "Layout": "string",
    "SurfaceCondition": "string",
    "Infrastructure": "string",
    "Localization": "string",
    "SchoolNear": "string",

    # 코드/번호 성격
    "RoadNumber": "string", 
    "RoadSecNumber": "string",
    "RoadMarkerId": "string",
           
    # 연속형 수치
    "RoadMarkerDistance": "float64",
    "StripWidth": "float64",
    "LaneWidth": "float64",

    # 갯수
    "LaneNumber": "Int64"
}

places_test = places_test.astype(places_type_reference)

In [53]:
# 변경된 info() 확인
places_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5779 entries, 0 to 5778
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   AccidentId          5779 non-null   string 
 1   RoadType            5779 non-null   string 
 2   RoadNumber          3549 non-null   string 
 3   RoadSecNumber       18 non-null     string 
 4   RoadLetter          265 non-null    string 
 5   Circulation         5740 non-null   string 
 6   LaneNumber          5728 non-null   Int64  
 7   SpecialLane         5721 non-null   string 
 8   Slope               5738 non-null   string 
 9   RoadMarkerId        4204 non-null   string 
 10  RoadMarkerDistance  4169 non-null   float64
 11  Layout              5741 non-null   string 
 12  StripWidth          1310 non-null   float64
 13  LaneWidth           1356 non-null   float64
 14  SurfaceCondition    5735 non-null   string 
 15  Infrastructure      5730 non-null   string 
 16  Locali

## 2) Train

In [54]:
# 데이터 확인
places_train.head(3)

,AccidentId,RoadType,RoadNumber,RoadSecNumber,RoadLetter,Circulation,LaneNumber,SpecialLane,Slope,RoadMarkerId,RoadMarkerDistance,Layout,StripWidth,LaneWidth,SurfaceCondition,Infrastructure,Localization,SchoolNear
0,201800000001,Departamental,Route 41,NaN,C,TwoWay,2.0,0,Flat,NaN,NaN,RightCurve,NaN,NaN,Normal,Unknown,Lane,0.0
1,201800000002,Communal,Dept-41,NaN,D,TwoWay,2.0,0,Flat,NaN,NaN,LeftCurve,NaN,NaN,Normal,Unknown,Lane,0.0
2,201800000003,Departamental,D39,NaN,D,TwoWay,2.0,0,Flat,NaN,NaN,NaN,NaN,NaN,Normal,Unknown,Lane,0.0


In [55]:
# 데이터 정보 확인
places_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54604 entries, 0 to 54603
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   AccidentId          54604 non-null  object 
 1   RoadType            54604 non-null  object 
 2   RoadNumber          34017 non-null  object 
 3   RoadSecNumber       153 non-null    float64
 4   RoadLetter          2701 non-null   object 
 5   Circulation         54219 non-null  object 
 6   LaneNumber          54125 non-null  float64
 7   SpecialLane         54116 non-null  object 
 8   Slope               54187 non-null  object 
 9   RoadMarkerId        39688 non-null  float64
 10  RoadMarkerDistance  39438 non-null  float64
 11  Layout              51133 non-null  object 
 12  StripWidth          12579 non-null  float64
 13  LaneWidth           12975 non-null  float64
 14  SurfaceCondition    51121 non-null  object 
 15  Infrastructure      51088 non-null  object 
 16  Loca

### 데이터 정합성 맞추기

#### AccidentId 컬럼
AccidentId는 아래와 같은 데이터 형식을 포함한다.
- '-' 하이픈 포함
- 공백 포함

In [56]:
# AccidentId 데이터 확인
places_train["AccidentId"].head(20)

0      201800000001
1      201800000002
2      201800000003
3      201800000004
4      201800000006
5     20-1800000008
6     20-1800000009
7      201800000010
8      201800000011
9      201800000012
10     201800000013
11     201800000014
12     201800000015
13     201800000016
14     201800000017
15     201800000018
16     201800000019
17     201800000020
18     201800000021
19     201800000022
Name: AccidentId, dtype: object

In [57]:
places_train["AccidentId"] = places_train["AccidentId"].str.replace("-","", regex = False)

In [58]:
places_train["AccidentId"] = places_train["AccidentId"].str.replace(" ","", regex = False)

In [59]:
# 최종 확인
places_train["AccidentId"].head(20)

0     201800000001
1     201800000002
2     201800000003
3     201800000004
4     201800000006
5     201800000008
6     201800000009
7     201800000010
8     201800000011
9     201800000012
10    201800000013
11    201800000014
12    201800000015
13    201800000016
14    201800000017
15    201800000018
16    201800000019
17    201800000020
18    201800000021
19    201800000022
Name: AccidentId, dtype: object

#### RoadType, RoadLetter, Circulation, Slope, Localization, SpecialLane
위 컬럼들은 아래와 같은 데이터 형식을 포함한다.
- 'Modified' 값이 Train에만 존재함

'Modified'는 Test에도 존재하지 않고 특성 정보를 담았다고 보기 어렵다.    
또한 컬럼 맥락에 맞지 않은 것이 대부분이기 때문에 데이터가 값을 유지 못한 상태로 보는것이 자연스럽다.    
데이터가 변경되면서 알 수 없는 값으로 변경 되었다고 볼 수 있어 이를 'Unknown'으로 변경한다.

In [60]:
# 변경할 컬럼 변수 저장
cols = ["RoadType", "RoadLetter", "Circulation", "Slope", "Localization", "SpecialLane"]

In [61]:
# Modified 값 확인하기
places_train[cols].isin(["Modified"]).sum()

RoadType        148
RoadLetter      140
Circulation     132
Slope           143
Localization    139
SpecialLane     118
dtype: int64

In [62]:
# 'Unknown' 수정
places_train[cols] = places_train[cols].replace("Modified", "Unknown")

In [63]:
# Modified 변경 확인
places_train[cols].isin(["Modified"]).sum()

RoadType        0
RoadLetter      0
Circulation     0
Slope           0
Localization    0
SpecialLane     0
dtype: int64

#### RoadNumber 컬럼
AccidentId는 아래와 같은 데이터 형식을 포함한다.
- '-' 하이픈 포함
- 문자열 포함

Test와 동일하게 숫자로만 변환하겠습니다.

In [64]:
# 데이터 확인
places_train["RoadNumber"].head(20)

0      Route 41
1       Dept-41
2           D39
3       Dept-39
4       Dept-39
5       Route -
6     Route 141
7          D641
8      Dept-641
9            41
10      Dept-22
11          947
12      Dept-22
13     Dept-917
14          NaN
15     Dept-917
16          145
17          NaN
18          NaN
19         D917
Name: RoadNumber, dtype: object

In [65]:
# 문자 제거하기 및 빈값 NaN 변환하기
places_train["RoadNumber"] = (
    places_train["RoadNumber"].str.replace(r"[^0-9]", "", regex = True)
    .replace("", np.nan)
)

In [66]:
# 최종 확인
places_train["RoadNumber"].head(20)

0      41
1      41
2      39
3      39
4      39
5     NaN
6     141
7     641
8     641
9      41
10     22
11    947
12     22
13    917
14    NaN
15    917
16    145
17    NaN
18    NaN
19    917
Name: RoadNumber, dtype: object

#### Layout, SurfaceCondition, Infrastructure
위 컬럼들은 아래와 같은 데이터 형식을 포함한다.
- 'Modified','-', '0' 값이 Train에만 존재함

'Modified'는 값이 존재했지만 수정된 상태의 값으로 값이 없다는 NaN 보다는 Unknown이 맞다고 판단했다. (앞의 이유와 같은 맥락)    
하지만 '-', '0'의 경우에는 애초에 값이 존재하지 않는 상태로 보는것이 가깝다고 생각해 'NaN'으로 변경한다.    
도로의 모양이나 표면 상태 등을 체크할 수 없기 때문에 0 이나 - 으로 입력했을 가능성이 더 높고 측정이 불가한 NaN과 가깝다고 판단했다.

In [67]:
# 변경할 컬럼 변수 저장
cols = ["Layout", "SurfaceCondition", "Infrastructure"]

In [68]:
# 값 확인하기
places_train[cols].isin(["Modified","-","0"]).sum()

Layout              1628
SurfaceCondition    1813
Infrastructure      1655
dtype: int64

In [69]:
# 'Unknown' 수정
places_train[cols] = places_train[cols].replace("Modified", "Unknown")

In [70]:
# '-' 수정
places_train[cols] = places_train[cols].replace("-", np.nan)

In [71]:
# '0' 수정
places_train[cols] = places_train[cols].replace("0", np.nan)

In [72]:
# 최종 변경 확인
places_train[cols].isin(["Modified","-","0"]).sum()

Layout              0
SurfaceCondition    0
Infrastructure      0
dtype: int64

### 중복 데이터 확인
Test의 중복 데이터를 확인시 중복 데이터는 존재하지 않지만 places의 경우 중복데이터가 존재한다.    
장소 정보로 사고건수에 대해 장소가 여러개 존재할 수 없어 AccidentId 기준으로 한 행이 존재해야한다.    

하지만 같은 행에 대한 중복과 AccidentId 기준 중복이 존재해 이를 처리한다.
- 값 > Unknown > NaN 순으로 선택된다.

In [73]:
# 중복 데이터 갯수 확인
print(f"중복된 데이터 갯수: {places_train.duplicated().sum()}")

중복된 데이터 갯수: 844


In [74]:
# 중복 데이터 삭제
places_train = places_train.drop_duplicates()

In [75]:
# 중복 제거 확인
print(f"중복된 데이터 갯수: {places_train.duplicated().sum()}")

중복된 데이터 갯수: 0


In [76]:
# AccidentId 기준 중복 데이터 갯수 확인
print(f"'AccidentId'기준 중복된 데이터 갯수: {places_train['AccidentId'].duplicated().sum()}")

'AccidentId'기준 중복된 데이터 갯수: 1756


In [77]:
# 'AccidentId'가 중복되는 데이터 조회하기 
places_train[places_train.duplicated(subset=['AccidentId'], keep=False)].sort_values('AccidentId').head(10)

,AccidentId,RoadType,RoadNumber,RoadSecNumber,RoadLetter,Circulation,LaneNumber,SpecialLane,Slope,RoadMarkerId,RoadMarkerDistance,Layout,StripWidth,LaneWidth,SurfaceCondition,Infrastructure,Localization,SchoolNear
16,201800000019,Departamental,145,NaN,NaN,TwoWay,2.0,0,Flat,NaN,NaN,Straight,NaN,NaN,NaN,Unknown,Lane,0.0
54511,201800000019,Departamental,NaN,NaN,NaN,TwoWay,2.0,0,Flat,NaN,NaN,Straight,NaN,NaN,Normal,Unknown,Lane,0.0
17,201800000020,Communal,NaN,NaN,NaN,TwoWay,2.0,0,Flat,NaN,NaN,LeftCurve,NaN,NaN,Normal,Unknown,Lane,0.0
52478,201800000020,Unknown,NaN,NaN,NaN,TwoWay,2.0,0,Flat,NaN,NaN,Unknown,NaN,NaN,Normal,Unknown,Lane,0.0
91,201800000100,Departamental,115,NaN,NaN,TwoWay,2.0,0,Flat,5.0,1000.0,Straight,NaN,NaN,Normal,Unknown,Lane,0.0
53530,201800000100,Departamental,115,NaN,NaN,TwoWay,2.0,0,Flat,5.0,1000.0,Unknown,NaN,NaN,Normal,NaN,Lane,0.0
129,201800000141,Departamental,938,NaN,NaN,TwoWay,2.0,0,Flat,NaN,NaN,Straight,NaN,NaN,Normal,Unknown,Lane,0.0
52875,201800000141,Departamental,938,NaN,NaN,TwoWay,2.0,0,Flat,NaN,NaN,Straight,NaN,NaN,Normal,Unknown,Unknown,0.0
136,201800000149,Departamental,938,NaN,NaN,TwoWay,2.0,0,Flat,NaN,NaN,Straight,NaN,NaN,NaN,Unknown,Lane,0.0
53664,201800000149,Departamental,938,NaN,NaN,TwoWay,2.0,0,Flat,NaN,NaN,Unknown,NaN,NaN,Normal,Unknown,Lane,0.0


In [78]:
# AccidentId 기준 중복 행 정규화 (값 > Unknown > NaN 우선 규칙)

def pick_value_first(s):
    s = s.dropna()
    if s.empty:
        return np.nan
    
    non_unknown = s[s != "Unknown"]
    if not non_unknown.empty:
        return non_unknown.iloc[0]
    
    return "Unknown"

places_train = (
    places_train
    .groupby("AccidentId", as_index=False)
    .agg({c: pick_value_first for c in places_train.columns if c != "AccidentId"})
)

In [79]:
# 중복 제거 확인
print(f"'AccidentId'기준 중복된 데이터 갯수: {places_train['AccidentId'].duplicated().sum()}")

'AccidentId'기준 중복된 데이터 갯수: 0


In [80]:
# 결합된 데이터 확인해보기
places_train[places_train['AccidentId'] == '201800000149']

,AccidentId,RoadType,RoadNumber,RoadSecNumber,RoadLetter,Circulation,LaneNumber,SpecialLane,Slope,RoadMarkerId,RoadMarkerDistance,Layout,StripWidth,LaneWidth,SurfaceCondition,Infrastructure,Localization,SchoolNear
136,201800000149,Departamental,938,NaN,NaN,TwoWay,2.0,0,Flat,NaN,NaN,Straight,NaN,NaN,Normal,Unknown,Lane,0.0


### 데이터 타입 기준 정리
- Test 와 동일하게 데이터 타입을 정리한다.

In [81]:
# Test와 동일하게 진행
places_type_reference = {
    "AccidentId": "string",

    # 범주형
    "RoadType": "string",
    "RoadLetter": "string",
    "Circulation": "string",
    "SpecialLane": "string",
    "Slope": "string",
    "Layout": "string",
    "SurfaceCondition": "string",
    "Infrastructure": "string",
    "Localization": "string",
    "SchoolNear": "string",

    # 코드/번호 성격
    "RoadNumber": "string",
    "RoadSecNumber": "string",
    "RoadMarkerId": "string",
    "LaneNumber": "string",        

    # 연속형 수치
    "RoadMarkerDistance": "float64",
    "StripWidth": "float64",
    "LaneWidth": "Int64"
}

places_train = places_train.astype(places_type_reference)

In [82]:
# 변경된 info() 확인
places_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52004 entries, 0 to 52003
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   AccidentId          52004 non-null  string 
 1   RoadType            52004 non-null  string 
 2   RoadNumber          32339 non-null  string 
 3   RoadSecNumber       149 non-null    string 
 4   RoadLetter          2563 non-null   string 
 5   Circulation         51639 non-null  string 
 6   LaneNumber          51548 non-null  string 
 7   SpecialLane         51539 non-null  string 
 8   Slope               51606 non-null  string 
 9   RoadMarkerId        37804 non-null  string 
 10  RoadMarkerDistance  37566 non-null  float64
 11  Layout              47449 non-null  string 
 12  StripWidth          11983 non-null  float64
 13  LaneWidth           12353 non-null  Int64  
 14  SurfaceCondition    47261 non-null  string 
 15  Infrastructure      47406 non-null  string 
 16  Loca

# 3. Users

Users는 사고에 관련된 인적 정보를 담고 있는 테이블이다.    
위와 동일하게 Test 데이터를 구조 기준으로 삼아 train 데이터의 중복, 자료 타입, 정합성을 단계적으로 처리한다.

In [83]:
users_train = pd.read_csv('./data/users_train.csv')
users_test = pd.read_csv('./data/users_test.csv')

## 1) Test

In [84]:
# 데이터 확인
users_test.head(3)

,AccidentId,VehicleId,Seat,Category,Gender,TripReason,SafetyDevice,SafetyDeviceUsed,PedestrianLocation,PedestrianAction,PedestrianCompany,BirthYear
0,201800000005,A01,1.0,Moped,Male,Leisure,Helmet,Yes,NaN,NaN,Unknown,2001.0
1,201800000005,B01,1.0,Car<=3.5T,Male,Leisure,SeatBelt,Yes,NaN,NaN,Unknown,1946.0
2,201800000007,A01,1.0,Car<=3.5T,Male,Other,SeatBelt,Yes,NaN,NaN,Unknown,1996.0


In [85]:
# 데이터 정보 확인
users_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13372 entries, 0 to 13371
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   AccidentId          13372 non-null  int64  
 1   VehicleId           13372 non-null  object 
 2   Seat                12024 non-null  float64
 3   Category            13372 non-null  object 
 4   Gender              13200 non-null  object 
 5   TripReason          10370 non-null  object 
 6   SafetyDevice        12774 non-null  object 
 7   SafetyDeviceUsed    11664 non-null  object 
 8   PedestrianLocation  1082 non-null   object 
 9   PedestrianAction    1106 non-null   object 
 10  PedestrianCompany   7621 non-null   object 
 11  BirthYear           13198 non-null  float64
dtypes: float64(2), int64(1), object(9)
memory usage: 1.2+ MB


In [86]:
# 중복 데이터 갯수 확인
print(f"중복된 데이터 갯수: {users_test.duplicated().sum()}")

중복된 데이터 갯수: 0


In [87]:
# AccidentId 기준 중복 데이터 갯수 확인
print(f"'AccidentId'기준 중복된 데이터 갯수: {users_test['AccidentId'].duplicated().sum()}")

'AccidentId'기준 중복된 데이터 갯수: 7593


In [88]:
# AccidentId + VehicleId + Seat 기준 중복 데이터 갯수 확인
print(f"'AccidentId+VehicleId+Seat' 기준 중복된 데이터 갯수: "
      f"{users_test.duplicated(subset=['AccidentId', 'VehicleId', 'Seat']).sum()}")

'AccidentId+VehicleId+Seat' 기준 중복된 데이터 갯수: 181


### 데이터 타입 기준 정의

- '년'도는 int64로 처리한다.
- 이외 다른 데이터는 카테고리나 범주형으로 string으로 변환
- accidentId, VehicleId 식별자로 string 변환한다.

In [89]:
# 데이터 타입 기준 정의
users_type_reference = {
    # 식별자
    "AccidentId": "int64",
    "VehicleId": "string",

    # 범주형
    "Category": "string",
    "Gender": "string",
    "TripReason": "string",
    "SafetyDevice": "string",
    "SafetyDeviceUsed": "string",
    "PedestrianLocation": "string",
    "PedestrianAction": "string",
    "PedestrianCompany": "string",

    # 출생년도
    "BirthYear": "Int64",
    "Seat": "Int64"
}

# 타입 적용
users_test = users_test.astype(users_type_reference)

In [90]:
# 데이터 변경 정보 확인
users_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13372 entries, 0 to 13371
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   AccidentId          13372 non-null  int64 
 1   VehicleId           13372 non-null  string
 2   Seat                12024 non-null  Int64 
 3   Category            13372 non-null  string
 4   Gender              13200 non-null  string
 5   TripReason          10370 non-null  string
 6   SafetyDevice        12774 non-null  string
 7   SafetyDeviceUsed    11664 non-null  string
 8   PedestrianLocation  1082 non-null   string
 9   PedestrianAction    1106 non-null   string
 10  PedestrianCompany   7621 non-null   string
 11  BirthYear           13198 non-null  Int64 
dtypes: Int64(2), int64(1), string(9)
memory usage: 1.2 MB


## 2) Train

In [91]:
# 데이터 확인 
users_train.head(3)

,AccidentId,VehicleId,Seat,Category,Gender,TripReason,SafetyDevice,SafetyDeviceUsed,PedestrianLocation,PedestrianAction,PedestrianCompany,BirthYear
0,201800000001,A01,Driver,Car<=3.5T,M,Leisure,SeatBelt,Yes,NaN,NaN,Unknown,1960.0
1,201800000001,B01,Front Left,Car<=3.5T,M,NaN,SeatBelt,1,NaN,NaN,Unknown,1928.0
2,201800000002,A01,Pilote,Auto,Homme,NaN,SeatBelt,True,NaN,NaN,Unknown,1947.0


In [92]:
# 데이터 정보 확인
users_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118412 entries, 0 to 118411
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   AccidentId          118412 non-null  int64 
 1   VehicleId           118412 non-null  object
 2   Seat                107007 non-null  object
 3   Category            118412 non-null  object
 4   Gender              116749 non-null  object
 5   TripReason          91137 non-null   object
 6   SafetyDevice        113062 non-null  object
 7   SafetyDeviceUsed    104246 non-null  object
 8   PedestrianLocation  8922 non-null    object
 9   PedestrianAction    9155 non-null    object
 10  PedestrianCompany   67696 non-null   object
 11  BirthYear           116759 non-null  object
dtypes: int64(1), object(11)
memory usage: 10.8+ MB


### 데이터 정합성 맞추기

#### Seat 컬럼

- 문자, 숫자 혼재
- 운전석을 여러형태의 문자로 표기함 (Driver, Pilote, Conducteur, front Left)

참고 : Front Left를 조수석으로 가정할 경우 하나의 차량 내에서 Seat 위치와 VehicleId가 비정상적으로 과도하게 중복으로 발생하므로 운전석으로 판단한다.

In [93]:
# Seat 고유 값 확인
users_train['Seat'].sort_values().unique()

array(['1', '2.0', '3.0', '4.0', '5.0', '6.0', '7.0', '8.0', '9.0',
       'Conducteur', 'Driver', 'Front Left', 'Pilote', nan], dtype=object)

In [94]:
# 매핑 코드 생성
seat_map = {
    'Driver' : '1.0',
    'Pilote' : '1.0',
    'Conducteur' : '1.0',
    'Front Left' : '1.0',
    '1' : "1.0",
}

In [95]:
# 매핑키 적용
users_train['Seat'] = users_train['Seat'].replace(seat_map)

In [96]:
# 변경 고유 값 확인
users_train['Seat'].sort_values().unique()

array(['1.0', '2.0', '3.0', '4.0', '5.0', '6.0', '7.0', '8.0', '9.0', nan],
      dtype=object)

#### Category 컬럼

- test와 정합성 불일치
- 오타, 단위 표기 오류
- 대소문자, 특수기호 차이
- 숫자 단위 표기에서 숫자와 알파벳 혼재

In [97]:
# Category 고유값 확인
users_train['Category'].sort_values().unique()

array(['Auto', 'Bicycle', 'Bicyde', 'Bus', 'Car', 'Car<=3.5T', 'Coach',
       'LargeCar+Trailer>3.5T', 'LargeCar>7.5T', 'LargeCar_3.5T-7.5T',
       'Light Vehicle', 'Light Vehide', 'Moped', 'Motorbike>125cm3',
       'Motorbike>l25cm3', 'Motorbike_50cm3-125cm3',
       'Motorbike_50cm3-l25cm3', 'Motorbike_5Ocm3-125cm3',
       'Motorbike_5Ocm3-l25cm3', 'Other', 'QuadBike<=50cm3',
       'QuadBike<=5Ocm3', 'QuadBike>50cm3', 'QuadBike>5Ocm3',
       'Quadricycle', 'Quadricyde', 'Scooter<50cm3', 'Scooter<5Ocm3',
       'Scooter>125cm3', 'Scooter>l25cm3', 'Scooter_50cm3-125cm3',
       'Scooter_50cm3-l25cm3', 'Scooter_5Ocm3-125cm3',
       'Scooter_5Ocm3-l25cm3', 'SpecialEngine', 'Tractor', 'Train',
       'Tramway', 'Truck', 'Truck+SemiTrailer', 'Utility', 'Voiture'],
      dtype=object)

In [98]:
# test 고유값 확인
users_test['Category'].sort_values().unique()

<StringArray>
[               'Bicycle',                    'Bus',              'Car<=3.5T',
                  'Coach',  'LargeCar+Trailer>3.5T',          'LargeCar>7.5T',
     'LargeCar_3.5T-7.5T',                  'Moped',       'Motorbike>125cm3',
 'Motorbike_50cm3-125cm3',                  'Other',        'QuadBike<=50cm3',
         'QuadBike>50cm3',            'Quadricycle',          'Scooter<50cm3',
         'Scooter>125cm3',   'Scooter_50cm3-125cm3',          'SpecialEngine',
                'Tractor',                  'Train',                'Tramway',
                  'Truck',      'Truck+SemiTrailer',                'Utility']
Length: 24, dtype: string

In [99]:
# 매핑 코드 만들기

vehicle_map = {
    # Car<=3.5T
    "Voiture": "Car<=3.5T",
    "Light Vehide": "Car<=3.5T",
    "Light Vehicle": "Car<=3.5T",
    "Car": "Car<=3.5T",
    "Auto": "Car<=3.5T",

    # Bicycle
    "Bicyde": "Bicycle",

    # Motorbike>125cm3
    "Motorbike>l25cm3": "Motorbike>125cm3",

    # Motorbike_50cm3-125cm3
    "Motorbike_5Ocm3-l25cm3": "Motorbike_50cm3-125cm3",
    "Motorbike_5Ocm3-125cm3": "Motorbike_50cm3-125cm3",
    "Motorbike_50cm3-l25cm3": "Motorbike_50cm3-125cm3",

    # QuadBike<=50cm3
    "QuadBike<=5Ocm3": "QuadBike<=50cm3",

    # QuadBike>50cm3
    "QuadBike>5Ocm3": "QuadBike>50cm3",

    # Quadricycle
    "Quadricyde": "Quadricycle",

    # Scooter<50cm3
    "Scooter<5Ocm3": "Scooter<50cm3",

    # Scooter>125cm3
    "Scooter>l25cm3": "Scooter>125cm3",

    # Scooter_50cm3-125cm3
    "Scooter_50cm3-l25cm3": "Scooter_50cm3-125cm3",
    "Scooter_5Ocm3-125cm3": "Scooter_50cm3-125cm3",
    "Scooter_5Ocm3-l25cm3": "Scooter_50cm3-125cm3"
    
}

In [100]:
# 매핑 코드 적용
users_train['Category'] = users_train['Category'].replace(vehicle_map)

In [101]:
# 결과 확인하기
users_train['Category'].sort_values().unique()

array(['Bicycle', 'Bus', 'Car<=3.5T', 'Coach', 'LargeCar+Trailer>3.5T',
       'LargeCar>7.5T', 'LargeCar_3.5T-7.5T', 'Moped', 'Motorbike>125cm3',
       'Motorbike_50cm3-125cm3', 'Other', 'QuadBike<=50cm3',
       'QuadBike>50cm3', 'Quadricycle', 'Scooter<50cm3', 'Scooter>125cm3',
       'Scooter_50cm3-125cm3', 'SpecialEngine', 'Tractor', 'Train',
       'Tramway', 'Truck', 'Truck+SemiTrailer', 'Utility'], dtype=object)

#### Gender 컬럼

- test와 정합성 불일치
- 약어, 불어, 영어, 숫자 등 다양한 표기

In [102]:
# 고유 값 확인
users_train['Gender'].unique()

array(['M', 'Homme', 'Male', '1', nan, 'H', '0', 'Femme', 'Female', 'F'],
      dtype=object)

In [103]:
# 매핑 코드 만들기
gender_map = {
    'M': 'Male',
    'H': 'Male',
    'Homme': 'Male',
    'Male': 'Male',
    '1': 'Male',

    'F': 'Female',
    'Femme': 'Female',
    'Female': 'Female',
    '0': 'Female',
}

In [104]:
# 매핑 값 적용
users_train['Gender'] = users_train['Gender'].replace(gender_map)

In [105]:
# 변경 고유 값 확인
users_train['Gender'].unique()

array(['Male', nan, 'Female'], dtype=object)

#### SafetyDeviceUsed 컬럼

- test와 정합성 불일치
- 영어, 불어, 문자, 숫자, 불타입 등 혼재

In [106]:
# 고유 값 확인
users_train['SafetyDeviceUsed'].unique()

array(['Yes', '1', 'True', nan, 'Oui', 'Y', 'Unknown', 'N', 'Non', '0',
       'False', 'No'], dtype=object)

In [107]:
# 매핑 함수 만들기
yes = ['Yes', '1', 'True', 'Oui', 'Y']
no = ['No', '0', 'False', 'N', 'Non']

def normalize_safety(value) :
    if value in yes :
        return 'Yes'
    elif value in no :
        return 'No'
    elif value == 'Unknown' :
        return 'Unknown'
    else :
        return value

In [108]:
# 매핑 값 적용
users_train['SafetyDeviceUsed'] = users_train['SafetyDeviceUsed'].apply(normalize_safety)

In [109]:
# 변경 고유 값 확인
users_train['SafetyDeviceUsed'].unique()

array(['Yes', nan, 'Unknown', 'No'], dtype=object)

#### BirthYear 컬럼

- 4자리 숫자 정합성 불일치
- REDACTED 발생
- 세자리 수 의미 알 수 없는 숫자 발생

**변환 기준 : 2018년에 발생한 데이터라는 점을 고려하여 두 자리 연도는 다음 기준으로 변환했다.**
- 0 ~ 18 → 2000~2018년생
- 19 ~ 99 → 1919~1999년생

In [110]:
users_train['BirthYear'].head(10)

0      1960.0
1      1928.0
2      1947.0
3        65.0
4        37.0
5         NaN
6    REDACTED
7      1982.0
8        11.0
9        40.0
Name: BirthYear, dtype: object

In [111]:
# Int64로 변환하여 연산하기 - 숫자 변환시 문자는 자동 결측처리
users_train['BirthYear'] = pd.to_numeric(users_train['BirthYear'], errors='coerce').astype("Int64")

In [112]:
# 범위 지정하기

# 2000~2018년
mask_0_18 = users_train['BirthYear'].between(0, 18)

# 1919~1999년
mask_19_99 = users_train['BirthYear'].between(19, 99)

In [113]:
# BirthYear를 4자리 연도로 정규화
users_train.loc[mask_0_18, 'BirthYear'] = users_train.loc[mask_0_18, 'BirthYear'] + 2000
users_train.loc[mask_19_99, 'BirthYear'] = users_train.loc[mask_19_99, 'BirthYear'] + 1900

In [114]:
# BirthYear 중 의미를 해석할 수 없는 3자리 값은 결측(<NA>)으로 처리
users_train[
    users_train["BirthYear"].notna() &
    ((users_train["BirthYear"] < 1000) | (users_train["BirthYear"] > 9999))
] = pd.NA

In [115]:
users_train['BirthYear'].head(10)

0    1960
1    1928
2    1947
3    1965
4    1937
5    <NA>
6    <NA>
7    1982
8    2011
9    1940
Name: BirthYear, dtype: Int64

### 중복데이터 확인

Users 데이터 셋은 Accidents 테이블과 1:N의 관계로 AccidentId 기준의 중복삭제는 진행하지 않았습니다.

In [116]:
# 중복 데이터 갯수 확인
print(f"중복된 데이터 갯수: {users_train.duplicated().sum()}")

중복된 데이터 갯수: 29


In [117]:
# AccidentId 기준 중복 데이터 갯수 확인
print(f"'AccidentId'기준 중복된 데이터 갯수: {users_train['AccidentId'].duplicated().sum()}")

'AccidentId'기준 중복된 데이터 갯수: 66409


In [118]:
# AccidentId + VehicleId + Seat 기준 중복 데이터 갯수 확인
print(f"'AccidentId+VehicleId+Seat' 기준 중복된 데이터 갯수: "
      f"{users_train.duplicated(subset=['AccidentId', 'VehicleId', 'Seat']).sum()}")

'AccidentId+VehicleId+Seat' 기준 중복된 데이터 갯수: 1143


### 데이터 타입 기준 정리
- Test 와 동일하게 데이터 타입을 정리한다.

In [119]:
users_train.head(3)

,AccidentId,VehicleId,Seat,Category,Gender,TripReason,SafetyDevice,SafetyDeviceUsed,PedestrianLocation,PedestrianAction,PedestrianCompany,BirthYear
0,2.018000e+11,A01,1.0,Car<=3.5T,Male,Leisure,SeatBelt,Yes,NaN,NaN,Unknown,1960
1,2.018000e+11,B01,1.0,Car<=3.5T,Male,NaN,SeatBelt,Yes,NaN,NaN,Unknown,1928
2,2.018000e+11,A01,1.0,Car<=3.5T,Male,NaN,SeatBelt,Yes,NaN,NaN,Unknown,1947


In [121]:
# 데이터 타입 기준 정의

users_type_reference = {
    # 식별자
    "AccidentId": "Int64", # 결측존재로 Int64 변경
    "VehicleId": "string",

    # 범주형
    "Category": "string",
    "Gender": "string",
    "TripReason": "string",
    "SafetyDevice": "string",
    "SafetyDeviceUsed": "string",
    "PedestrianLocation": "string",
    "PedestrianAction": "string",
    "PedestrianCompany": "string",

    # 출생년도
    "BirthYear": "Int64"
}

# 타입 적용
users_train = users_train.astype(users_type_reference)

# Seat는 따로 Int적용
users_train["Seat"] = pd.to_numeric(users_train["Seat"], errors="coerce").astype("Int64")

In [122]:
# 데이터 타입 변경 확인
users_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118412 entries, 0 to 118411
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   AccidentId          118382 non-null  Int64 
 1   VehicleId           118382 non-null  string
 2   Seat                106992 non-null  Int64 
 3   Category            118382 non-null  string
 4   Gender              116719 non-null  string
 5   TripReason          91118 non-null   string
 6   SafetyDevice        113037 non-null  string
 7   SafetyDeviceUsed    104230 non-null  string
 8   PedestrianLocation  8907 non-null    string
 9   PedestrianAction    9140 non-null    string
 10  PedestrianCompany   67673 non-null   string
 11  BirthYear           114317 non-null  Int64 
dtypes: Int64(3), string(9)
memory usage: 11.2 MB


In [123]:
users_train.head(3)

,AccidentId,VehicleId,Seat,Category,Gender,TripReason,SafetyDevice,SafetyDeviceUsed,PedestrianLocation,PedestrianAction,PedestrianCompany,BirthYear
0,201800000001,A01,1,Car<=3.5T,Male,Leisure,SeatBelt,Yes,<NA>,<NA>,Unknown,1960
1,201800000001,B01,1,Car<=3.5T,Male,<NA>,SeatBelt,Yes,<NA>,<NA>,Unknown,1928
2,201800000002,A01,1,Car<=3.5T,Male,<NA>,SeatBelt,Yes,<NA>,<NA>,Unknown,1947


# 4. Vehicles

## 1) Test

In [124]:
# 데이터 확인
vehicles_test.head(3)

,AccidentId,VehicleId,Direction,Category,PassengerNumber,FixedObstacle,MobileObstacle,ImpactPoint,Maneuver
0,201800000005,A01,Unknown,Moped,0,NaN,Vehicle,RightFront,PassLeft
1,201800000005,B01,Unknown,Car<=3.5T,0,NaN,Vehicle,LeftFront,Park
2,201800000007,A01,Unknown,Car<=3.5T,0,Post,NaN,LeftSide,NaN


In [125]:
# 데이터 정보 확인
vehicles_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9893 entries, 0 to 9892
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   AccidentId       9893 non-null   int64 
 1   VehicleId        9893 non-null   object
 2   Direction        9880 non-null   object
 3   Category         9893 non-null   object
 4   PassengerNumber  9893 non-null   int64 
 5   FixedObstacle    1324 non-null   object
 6   MobileObstacle   8240 non-null   object
 7   ImpactPoint      9357 non-null   object
 8   Maneuver         9131 non-null   object
dtypes: int64(2), object(7)
memory usage: 695.7+ KB


In [126]:
# 중복 데이터 갯수 확인
print(f"중복된 데이터 갯수: {vehicles_test.duplicated().sum()}")

중복된 데이터 갯수: 0


In [127]:
# AccidentId 기준 중복 데이터 갯수 확인
print(f"'AccidentId'기준 중복된 데이터 갯수: {vehicles_test['AccidentId'].duplicated().sum()}")

'AccidentId'기준 중복된 데이터 갯수: 4114


### 데이터 타입 기준 정의

- accidentId, VehicleId 식별자로 string 변환한다.
- PassengerNumber는 차량에 탑승인 수로 int 변환한다.
- 이외 다른 데이터는 카테고리나 범주형으로 string으로 변환한다.

In [128]:
# 데이터 타입 기준 정의
vehicles_type_reference = {
    # 식별자
    "AccidentId": "int64",
    "VehicleId": "string",

    # 범주형
    "Direction": "string",
    "Category": "string",
    "FixedObstacle": "string",
    "MobileObstacle": "string",
    "ImpactPoint": "string",
    "Maneuver": "string",
    
    # 탑승자 수
    "PassengerNumber": "Int64"
}

# 타입 적용
vehicles_test = vehicles_test.astype(vehicles_type_reference)

In [129]:
# 데이터 변경 정보 확인
vehicles_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9893 entries, 0 to 9892
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   AccidentId       9893 non-null   int64 
 1   VehicleId        9893 non-null   string
 2   Direction        9880 non-null   string
 3   Category         9893 non-null   string
 4   PassengerNumber  9893 non-null   Int64 
 5   FixedObstacle    1324 non-null   string
 6   MobileObstacle   8240 non-null   string
 7   ImpactPoint      9357 non-null   string
 8   Maneuver         9131 non-null   string
dtypes: Int64(1), int64(1), string(7)
memory usage: 705.4 KB


## 2) Train

In [130]:
# 데이터 확인
vehicles_train.head(3)

,AccidentId,VehicleId,Direction,Category,PassengerNumber,FixedObstacle,MobileObstacle,ImpactPoint,Maneuver
0,201800000001,V01,Unk,Car<=3.5T,0,NaN,Vehicle,Front Right,Turning Left
1,201800000001,Car_01,?,Car<=3.5T,0,NaN,Blue Ford,LeftFront,NoDirectionChange
2,201800000002,V01,Unk,Car<=3.5T,0,NaN,Pedestrian,NaN,NoDirectionChange


In [131]:
# 데이터 정보 확인
vehicles_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88983 entries, 0 to 88982
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   AccidentId       88983 non-null  int64 
 1   VehicleId        88983 non-null  object
 2   Direction        88864 non-null  object
 3   Category         88983 non-null  object
 4   PassengerNumber  88983 non-null  int64 
 5   FixedObstacle    12792 non-null  object
 6   MobileObstacle   73314 non-null  object
 7   ImpactPoint      84125 non-null  object
 8   Maneuver         81811 non-null  object
dtypes: int64(2), object(7)
memory usage: 6.1+ MB


### 데이터 정합성 맞추기

#### vehicleId, Category
Vehicles의 `VehicleId, Category`의 정합성이 너무 깨져있어서 복원하기 힘든 상태로 삭제처리 하겠습니다.    
VehicleId와 Category는 Users 데이터 셋에서도 동일하게 있어 Users 데이터 셋의 데이터를 사용하겠습니다.

In [132]:
# VehicleId 삭제(Drop) - test, train 모두 삭제
vehicles_train = vehicles_train.drop(columns=["VehicleId"])
vehicles_test = vehicles_test.drop(columns=["VehicleId"])

In [133]:
# Category 삭제(Drop) - test, train 모두 삭제
vehicles_train = vehicles_train.drop(columns=["Category"])
vehicles_test = vehicles_test.drop(columns=["Category"])

In [134]:
# 삭제 확인
vehicles_train.head(3)

,AccidentId,Direction,PassengerNumber,FixedObstacle,MobileObstacle,ImpactPoint,Maneuver
0,201800000001,Unk,0,NaN,Vehicle,Front Right,Turning Left
1,201800000001,?,0,NaN,Blue Ford,LeftFront,NoDirectionChange
2,201800000002,Unk,0,NaN,Pedestrian,NaN,NoDirectionChange


#### Direction 컬럼

- 특수문자 
- 프랑스어, 영어 혼재

In [135]:
# 고유값 확인
vehicles_train['Direction'].unique()

array(['Unk', '?', 'Unknown', 'Inconnu', 'Increasing', 'Decreasing', nan],
      dtype=object)

In [136]:
# 매핑키 생성
Direction_map = {
    "Unk": "Unknown",
    "?": "Unknown",
    "Inconnu": "Unknown"
}

# 타입 적용
vehicles_train['Direction'] = vehicles_train['Direction'].replace(Direction_map)

In [137]:
# 고유값 변경 확인
vehicles_train['Direction'].unique()

array(['Unknown', 'Increasing', 'Decreasing', nan], dtype=object)

#### MobileObstacle 컬럼

- 차량의 세부 모델이 기재되어있다.
- 차량의 기종들은 모두 vehicles로 바꿔 정합성을 일치한다.

In [138]:
# 고유값 확인
vehicles_train['MobileObstacle'].unique()

array(['Vehicle', 'Blue Ford', 'Pedestrian', nan, 'Other', 'Red Toyota',
       'Motorcycle', 'Truck', 'White Van', 'WildAnimal', 'DomesticAnimal',
       'RailedVehicle'], dtype=object)

In [139]:
# 매핑키 생성
MobileObstacle_map = {
    "White Van": "Vehicle",
    "Truck": "Vehicle",
    "Red Toyota": "Vehicle",
    "Motorcycle": "Vehicle",
    "Blue Ford": "Vehicle"
}

# 타입 적용
vehicles_train['MobileObstacle'] = vehicles_train['MobileObstacle'].replace(MobileObstacle_map)

In [140]:
# 변경된 고유값 확인
vehicles_train['MobileObstacle'].unique()

array(['Vehicle', 'Pedestrian', nan, 'Other', 'WildAnimal',
       'DomesticAnimal', 'RailedVehicle'], dtype=object)

#### ImpactPoint 컬럼

- 같은 의미의 값이 영어, 불어, 약어 등 존재 
- test와 정합성 일치하기

In [141]:
# 고유값 확인
vehicles_train['ImpactPoint'].unique()

array(['Front Right', 'LeftFront', nan, 'Front', 'LeftSide', 'RightSide',
       'Avant Droit', 'Back', 'LeftBack', 'Right Front', 'Multiple',
       'RightBack', 'RF'], dtype=object)

In [142]:
# 매핑키 생성
ImpactPoint_map = {
    "Right Front": "RightFront",
    "Front Right": "RightFront",
    "RF": "RightFront",
    "Avant Droit": "RightFront"
}

# 타입 적용
vehicles_train['ImpactPoint'] = vehicles_train['ImpactPoint'].replace(ImpactPoint_map)

In [143]:
# 변경 고유값 확인
vehicles_train['ImpactPoint'].unique()

array(['RightFront', 'LeftFront', nan, 'Front', 'LeftSide', 'RightSide',
       'Back', 'LeftBack', 'Multiple', 'RightBack'], dtype=object)

#### ImpactPoint 컬럼

- 오타, 다국어, 약어 등 혼재
- test와 정합성 일치하기

In [144]:
# 고유값 확인
vehicles_train['Maneuver'].unique()

array(['Turning Left', 'NoDirectionChange', 'Virage Gauche', 'Parked',
       'Avoidance', nan, 'PassLeft', 'TL', 'Turn To Left', 'WrongWay',
       'Stopped', 'Tuming Left', 'SameDirectionOrLane', 'SwerveToLeft',
       'Turring Left', 'Turned Left', 'Insertion', 'CrossLane',
       'TurnToRight', 'SwerveToRight', 'UTurnInLane', 'Park',
       'ChangeToRightLane', 'ChangeToLeftLane', 'Reverse', 'CrossStrip',
       'PassRight', 'BetweenLanes', 'TumToRight', 'Tum To Left',
       'Tumed Left', 'OpenDoor', 'BusLaneWrongDirection',
       'BusLaneSameDirection', 'UTumInLane'], dtype=object)

In [145]:
# 매핑키 생성

maneuver_map = {
    # U-Turn 오타
    'UTumInLane': 'UTurnInLane',

    # 좌회전(Left Turn) 관련
    'Virage Gauche': 'TurnToLeft',
    'Turring Left': 'TurnToLeft',
    'Turning Left': 'TurnToLeft',
    'Turned Left': 'TurnToLeft',
    'Turn To Left': 'TurnToLeft',
    'Tuming Left': 'TurnToLeft',
    'Tumed Left': 'TurnToLeft',
    'Tum To Left': 'TurnToLeft',
    'TL': 'TurnToLeft',

    # 우회전(Right Turn) 관련
    'TumToRight': 'TurnToRight' 
}

# 타입 적용
vehicles_train['Maneuver'] = vehicles_train['Maneuver'].replace(maneuver_map)

In [146]:
# 변경 고유값 확인
vehicles_train['Maneuver'].unique()

array(['TurnToLeft', 'NoDirectionChange', 'Parked', 'Avoidance', nan,
       'PassLeft', 'WrongWay', 'Stopped', 'SameDirectionOrLane',
       'SwerveToLeft', 'Insertion', 'CrossLane', 'TurnToRight',
       'SwerveToRight', 'UTurnInLane', 'Park', 'ChangeToRightLane',
       'ChangeToLeftLane', 'Reverse', 'CrossStrip', 'PassRight',
       'BetweenLanes', 'OpenDoor', 'BusLaneWrongDirection',
       'BusLaneSameDirection'], dtype=object)

### 데이터 타입 기준 정의
- 삭제된 일부 컬럼 제외 test와 동일한 type 적용한다.

In [147]:
# 데이터 타입 기준 정의

vehicles_type_reference = {
    # 식별자
    "AccidentId": "int64",

    # 범주형
    "Direction": "string",
    "FixedObstacle": "string",
    "MobileObstacle": "string",
    "ImpactPoint": "string",
    "Maneuver": "string",
    
    # 탑승자 수
    "PassengerNumber": "Int64"
}

# 타입 적용
vehicles_train = vehicles_train.astype(vehicles_type_reference)

In [148]:
# 데이터 변경 정보 확인
vehicles_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88983 entries, 0 to 88982
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   AccidentId       88983 non-null  int64 
 1   Direction        88864 non-null  string
 2   PassengerNumber  88983 non-null  Int64 
 3   FixedObstacle    12792 non-null  string
 4   MobileObstacle   73314 non-null  string
 5   ImpactPoint      84125 non-null  string
 6   Maneuver         81811 non-null  string
dtypes: Int64(1), int64(1), string(5)
memory usage: 4.8 MB


### 중복 데이터 확인

In [149]:
# 중복 데이터 갯수 확인
print(f"중복된 데이터 갯수: {vehicles_train.duplicated().sum()}")

중복된 데이터 갯수: 2505


In [150]:
# 중복 데이터 삭제 하기
vehicles_train = vehicles_train.drop_duplicates()

In [151]:
# AccidentId 기준 중복 데이터 갯수 확인
print(f"'AccidentId'기준 중복된 데이터 갯수: {vehicles_train['AccidentId'].duplicated().sum()}")

'AccidentId'기준 중복된 데이터 갯수: 34474


# 최종 데이터 셋 저장하기

In [152]:
users_train

,AccidentId,VehicleId,Seat,Category,Gender,TripReason,SafetyDevice,SafetyDeviceUsed,PedestrianLocation,PedestrianAction,PedestrianCompany,BirthYear
0,201800000001,A01,1,Car<=3.5T,Male,Leisure,SeatBelt,Yes,<NA>,<NA>,Unknown,1960
1,201800000001,B01,1,Car<=3.5T,Male,<NA>,SeatBelt,Yes,<NA>,<NA>,Unknown,1928
2,201800000002,A01,1,Car<=3.5T,Male,<NA>,SeatBelt,Yes,<NA>,<NA>,Unknown,1947
3,201800000002,A01,<NA>,Car<=3.5T,Male,<NA>,Helmet,<NA>,OnLane<=OnSidewalk0mCrossing,Crossing,Alone,1965
4,201800000003,A01,1,Motorbike>125cm3,Male,Leisure,Helmet,Yes,<NA>,<NA>,Unknown,1937
...,...,...,...,...,...,...,...,...,...,...,...,...
118407,201800057781,A01,<NA>,Car<=3.5T,Male,Other,ChildrenDevice,<NA>,OnCrossingWithoutLights,Crossing,Alone,1975
118408,201800057782,A01,1,Car<=3.5T,Female,Other,SeatBelt,Yes,<NA>,<NA>,<NA>,1993
118409,201800057782,B01,1,Motorbike_50cm3-125cm3,Male,Other,Helmet,Yes,<NA>,<NA>,<NA>,1974
118410,201800057783,A01,1,Utility,Female,Other,SeatBelt,Yes,<NA>,<NA>,<NA>,1934


In [153]:
# Accidents

accidents_test.to_csv("./data/preprocessed/Accidents_test_preprocessed.csv",
                       index=False,
                       encoding="utf-8-sig")

accidents_train.to_csv("./data/preprocessed/Accidents_train_preprocessed.csv",
                       index=False,
                       encoding="utf-8-sig")

In [154]:
# Places

places_test.to_csv("./data/preprocessed/places_test_preprocessed.csv",
                       index=False,
                       encoding="utf-8-sig")

places_train.to_csv("./data/preprocessed/places_train_preprocessed.csv",
                       index=False,
                       encoding="utf-8-sig")

In [155]:
# Users

users_test.to_csv("./data/preprocessed/users_test_preprocessed.csv",
                       index=False,
                       encoding="utf-8-sig")

users_train.to_csv("./data/preprocessed/users_train_preprocessed.csv",
                       index=False,
                       encoding="utf-8-sig")

In [156]:
# Vehicles

vehicles_test.to_csv("./data/preprocessed/vehicles_test_preprocessed.csv",
                       index=False,
                       encoding="utf-8-sig")

vehicles_train.to_csv("./data/preprocessed/vehicles_train_preprocessed.csv",
                       index=False,
                       encoding="utf-8-sig")